In [1]:
from utils.import_functions import get_data_from_rdw
from utils.conversion_functions import convert_list_to_df, clean_df
from utils.export_functions import export_to_csv

from datetime import date

In [2]:
# import the cars from the rdw
selected_brand = "bmw"
cars_list = get_data_from_rdw(selected_brand)

✅ Found 1000 cars for brand bmw


In [3]:
# convert the list to a pandas DataFrame
cars_df = convert_list_to_df(cars_list, "kenteken", 
                                        "handelsbenaming", 
                                        "catalogusprijs",
                                        "datum_tenaamstelling", 
                                        "aantal_cilinders", 
                                        "aantal_zitplaatsen",
                                        handelsbenaming="model", 
                                        catalogusprijs="prijs")

/Users/arietwigt/Projects/python_landstede/utils/conversion_functions.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns=kwargs, inplace=True)


In [4]:
# apply data cleaning
cars_df_clean = clean_df(cars_df, "prijs", 
                                  "aantal_cilinders",
                                  "aantal_zitplaatsen",
                                  prijs=float,
                                  aantal_cilinders=int,
                                  aantal_zitplaatsen=int,
                                  datum_tenaamstelling=date)

Convert prijs to data type <class 'float'>
Convert aantal_cilinders to data type <class 'int'>
Convert aantal_zitplaatsen to data type <class 'int'>
Convert datum_tenaamstelling to data type <class 'datetime.date'>


In [5]:
cars_df_clean.head(10)

,kenteken,model,prijs,datum_tenaamstelling,aantal_cilinders,aantal_zitplaatsen
20,00GBR2,3ER REIHE,35452.0,2010-05-20,4,5
21,00GDB4,1ER REIHE,30427.0,2021-11-19,4,4
22,00GDT8,5ER REIHE,58168.0,2019-07-29,4,5
23,00GFB1,1ER REIHE,55333.0,2008-06-06,4,4
25,00GFZ3,1ER REIHE,37411.0,2022-08-20,4,5
28,00GHD3,3ER REIHE,32760.0,2023-05-12,4,5
29,00GHK7,3ER REIHE,45136.0,2018-02-10,4,4
30,00GHP7,1ER REIHE,30288.0,2020-06-19,4,5
31,00GHX2,5ER REIHE,88957.0,NaT,6,5
32,00GJF6,X REIHE,57396.0,2024-01-12,4,5


In [6]:
# technical information of DataFrame
cars_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 20 to 999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   kenteken              607 non-null    object        
 1   model                 607 non-null    object        
 2   prijs                 607 non-null    float64       
 3   datum_tenaamstelling  545 non-null    datetime64[ns]
 4   aantal_cilinders      607 non-null    int64         
 5   aantal_zitplaatsen    607 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 33.2+ KB


In [7]:
# show statistical information
cars_df_clean.describe()

,prijs,datum_tenaamstelling,aantal_cilinders,aantal_zitplaatsen
count,607.000000,545,607.000000,607.000000
mean,52562.599671,2019-10-05 21:55:48.990825728,4.497529,4.823723
min,26199.000000,2008-06-06 00:00:00,4.000000,2.000000
25%,35476.000000,2017-05-08 00:00:00,4.000000,5.000000
50%,45895.000000,2020-10-02 00:00:00,4.000000,5.000000
75%,63350.500000,2022-12-29 00:00:00,4.000000,5.000000
max,181754.000000,2024-03-02 00:00:00,8.000000,7.000000
std,22922.843099,NaN,0.945500,0.494422


In [8]:
# 
cars_df_clean['prijs'].std()

22922.843099016143

In [9]:
# non-method chaining
name = "arie"
name = name.upper()
name = name.replace("A", "U")
name

'URIE'

In [10]:
# methond chaining
name = "arie"

name.upper().replace("A", "U")


'URIE'

In [11]:
# aggregation pipeline ()
cars_df_aggr = (cars_df_clean
                .groupby("model")
                .agg({
                    'prijs': 'mean',
                    'aantal_cilinders': 'max'})
                .rename(columns={'prijs': 'gemiddelde_prijs',
                                 'aantal_cilinders': 'max_cilinders'})
                .sort_values(by="gemiddelde_prijs", ascending=False)
                .query("gemiddelde_prijs > 100000 & max_cilinders >= 8")
                .reset_index()
)

cars_df_aggr

,model,gemiddelde_prijs,max_cilinders
0,ACTIVEHYBRID 7 L,181754.000000,8
1,M5,137852.000000,8
2,6ER REIHE,126722.666667,8
3,645 CI,118458.000000,8
4,550I,111059.000000,8


In [12]:
# export the DataFrame to csv
export_to_csv(cars_df_aggr, selected_brand)

✅ Saved file to data/bmw.csv
